# Анализ тематик текстов при помощи BERTopic

In [ ]:
%%capture
!pip install bertopic

In [1]:
import unicodedata
import pandas as pd
from bertopic import BERTopic
import transformers
import torch
import numpy as np
from nltk.corpus import stopwords
import nltk
from bertopic.representation import KeyBERTInspired

ModuleNotFoundError: No module named 'bertopic'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
df = pd.read_csv('/kaggle/input/mail-news/all_news')
df.head()

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# эмбеддинги
embedding_model = transformers.pipeline("feature-extraction", model="cointegrated/rubert-tiny2", device=device)

# понижение размерности
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# кластеризация
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean',
                        cluster_selection_method='eom', prediction_data=True)

# векторизация
list_stop_words = stopwords.words('russian')
vectorizer_model = CountVectorizer(stop_words=list_stop_words, min_df=2, ngram_range=(1, 2))

# извлечение ключевых слов
keybert_model = KeyBERTInspired()
representation_model = {
    "KeyBERT": keybert_model
}

In [ ]:
embedding_model = transformers.pipeline("feature-extraction", model="cointegrated/rubert-tiny2", device=device)

topic_model = BERTopic(embedding_model=embedding_model,
                       vectorizer_model = vectorizer_model,
                       representation_model = representation_model,
                       language="multilingual",
                       verbose=True)

In [ ]:
%%time
topics, probs = topic_model.fit_transform(df['Article Text'])

In [25]:
topics = topic_model.get_topic_info()
topics.head()

,Topic,Count,Name,Representation,KeyBERT,Representative_Docs
0,-1,32712,-1_это_также_которые_время,"[это, также, которые, время, могут, лет, года,...","[врач, врачи, пациентов, заболеваний, лечения,...","[Надеюсь, каждый родитель узнал хотя бы одну ч..."
1,0,3403,0_фильма_сериала_актер_роли,"[фильма, сериала, актер, роли, роль, фильм, фи...","[игра престолов, сериала, сериале, фильм, филь...",[Создатели фэнтезийной саги «Игра престолов» с...
2,1,1082,1_мозга_ученые_исследования_университета,"[мозга, ученые, исследования, университета, ис...","[исследования проведенного, результаты исследо...","[Между тем, как вас зовут, и тем, как вы выгля..."
3,2,1038,2_врачи_операции_операцию_опухоль,"[врачи, операции, операцию, опухоль, больницу,...","[пациентке, клиническом, клиническом случае, х...",[Об этом клиническом случае рассказали в журна...
4,3,1032,3_диетолог_врач_продукты_врач диетолог,"[диетолог, врач, продукты, врач диетолог, упот...","[врач диетолог, витамины, витаминов, диетолог,...","[Мы с детства знаем, что овощи полезны для здо..."


In [34]:
topic_model.visualize_barchart(top_n_topics=10)

In [35]:
topic_model.visualize_hierarchy()

In [20]:
topic_model.visualize_topics()